In [2]:
# DataFrame
import pandas as pd

In [3]:
dataset_path = "dataset/training.1600000.processed.noemoticon.csv"
print("Open file:", dataset_path)
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
df = pd.read_csv(dataset_path, encoding="ISO-8859-1" , names=DATASET_COLUMNS)
print("Dataset size:", len(df))

Open file: dataset/training.1600000.processed.noemoticon.csv
Dataset size: 1600000


In [37]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [38]:
from collections import Counter
train_size, test_size = 40000, 10000
train = df[:train_size].append(df[800000 : 800000 + train_size])
test = df[train_size : train_size + test_size].append(df[800000 + train_size:800000 + train_size + test_size])

print(Counter(train.target))
print(Counter(test.target))

Counter({0: 40000, 4: 40000})
Counter({0: 10000, 4: 10000})


In [39]:
train.text = train.text.apply(lambda x: preprocess(x))
test.text = test.text.apply(lambda x: preprocess(x))

In [40]:
X_train = train["text"].tolist() 
y_train = train["target"].tolist() 
X_test = test["text"].tolist() 
y_test = test["target"].tolist() 

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
print(vectorizer.get_feature_names())
X_train = X_train.toarray()
print(X_train.shape)

full_vocab = vectorizer.vocabulary_
vectorizer_test = CountVectorizer(vocabulary=full_vocab)
X_test = vectorizer_test.fit_transform(X_test)
X_test = X_test.toarray()
print(X_test.shape)


['00', '000', '0000', '0007', '000kg', '001', '002', '003', '004', '00am', '00h', '00pm', '01', '01614948343', '018', '02', '026', '026ers', '02am', '02r', '03', '04', '05', '051', '0515', '05am', '05pm', '06', '0640', '06am', '06pm', '07', '0730', '0752', '076', '07am', '07raus09', '08', '0800', '08am', '09', '095', '09pm', '0b2', '0g', '0h', '0ht', '0mg', '0n', '0r', '0x', '10', '100', '1000', '10000', '1000000', '1000d', '1000fps', '1000th', '1000w', '1001', '1003', '100deg', '100degrees', '100gb', '100k', '100m2', '100mb', '100miler', '100mph', '100plus', '100th', '100to', '100x', '101', '1010', '1012', '101th', '102', '1029', '103', '1030', '103rd', '105', '10570', '105f', '106', '1066', '107', '108', '1080p', '1099', '10am', '10e', '10ft', '10h30', '10hrs', '10ish', '10k', '10km', '10lbs', '10mb', '10mbps', '10mila', '10min', '10minutes', '10mph', '10oz', '10p', '10pack', '10pm', '10pt', '10pts', '10s', '10th', '10x', '10year', '10yrs', '11', '110', '1100', '1100dls', '1108', '11

(80000, 47548)
(20000, 47548)


In [42]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print(X_test.shape[0], (y_test == y_pred).sum())

20000 10415


In [32]:
# y_pred = gnb.predict(X_test)
print(X_test.shape[0], (y_test == y_pred).sum())

20000 10415
